In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as tf
import time

import numpy as np

from model import CRNN_Network

In [2]:
# Class for loading in dataset

import io
import pandas as pd
class BachDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file: str, root_dir: str):
        """
        Arguments:
            csv_file (string): Path to the csv file.
            root_dir (string): Directory with all the images.
        """
        self.data = pd.read_csv(root_dir + csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int = 0):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img = io.imread(self.root_dir + f'invent{idx+1}32.cropped.png')
        ly = torchvision.io.read_file(self.root_dir + f'invent{idx+1}.ly')
        
        sample = {'img': img, 'ly': ly}

        return sample


In [4]:
from quant_parse import quant_parse

# Data normalization: get optimal quantization (1, 2, 4, 8, 16, 32) from .ly file.
# This has been done pre-training, but this block shows what code was run:
"""
for i in range(1, 16):
    q = quant_parse(i)
    print(q)
"""

q = quant_parse(2)
q

32

In [5]:
batch_size = 128

root_dir = './outputs/bach/invent/'

trainset = BachDataset(csv_file='dataset.csv', root_dir=root_dir)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
from torch.nn import Module
from torch.optim import Optimizer 
from torch import Tensor
from torch.utils.data import DataLoader

net = CRNN_Network()
net = net.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)

TypeError: unsupported operand type(s) for %: 'ellipsis' and 'int'

In [ ]:
# Training Loop for CNN
def train(crnn: CRNN_Network, num_epochs: int):
    for e in range(num_epochs):

        running_loss: float = 0.

        for i, (img, ly) in trainloader:
            images, lys = img.to(device), ly.to(device)

            optimizer.zero_grad()

            outputs = crnn(images)
            loss = criterion(outputs, lys)
            loss.backward()
            optimizer.step()

            predicted = torch.argmax(outputs, 1).to(device)

            running_loss += loss.item()

            print_freq = 100
            if i % print_freq == (print_freq - 1):
                print(f'[{e + 1}, {i + 1:5d}] loss: {running_loss / print_freq:.3f} acc: {100 * running_correct / running_total:.2f} time: {time.time() - start_time:.2f}')
                running_loss, running_correct, running_total = 0.0, 0.0, 0.0
                start_time = time.time()

    print("Training complete!")


In [ ]:
num_epochs = 45
init_lr = 0.001
decay_epochs = 15
task = 'rotation'
train(net, criterion, optimizer, num_epochs, decay_epochs, init_lr, task)
torch.save(net.state_dict(), f"./model-ep{num_epochs}-initlr_{init_lr}-decay{decay_epochs}-{task}.pth")